In [12]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
pd.set_option('display.max_rows', None)
import datetime
from plotly.subplots import make_subplots

#### [Dataset](https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i)

#####  Column Description


| Column Name | Description |
| ----------- | ----------- |
| CMPLNT_NUM | Randomly generated persistent ID for each complaint | 
| ADDR_PCT_CD | The precinct in which the incident occurred BORO The name of the borough in which the incident occurred |
| CMPLNT_FR_DT | Exact date of occurrence for the reported event (or starting date of occurrence, if CMPLNT_TO_DT exists) |
| CMPLNT_FR_TM | Exact time of occurrence for the reported event (or starting time of occurrence, if CMPLNT_TO_TM exists) |
| CMPLNT_TO_DT | Ending date of occurrence for the reported event, if exact time of occurrence is unknown |
| CMPLNT_TO_TM | Ending time of occurrence for the reported event, if exact time of occurrence is unknown |
| CRM_ATPT_CPTD_CD | Indicator of whether crime was successfully completed or attempted, but failed or was interrupted prematurely |
| HADEVELOPT | Name of NYCHA housing development of occurrence, if applicable |
| HOUSING_PSA | Development Level Code |
| JURISDICTION_CODE | Jurisdiction responsible for incident. Either internal, like Police(0), Transit(1), and Housing(2); or external(3), like Correction, Port Authority, etc. |
| JURIS_DESC |  Description of the jurisdiction code |
| KY_CD | Three digit offense classification code |
| LAW_CAT_CD | Level of offense: felony, misdemeanor, violation |
| LOC_OF_OCCUR_DESC | Specific location of occurrence in or around the premises; inside, opposite of, front of, rear of |
| OFNS_DESC | Description of offense corresponding with key code |
| PARKS_NM | Name of NYC park, playground or greenspace of occurrence, if applicable (state parks are not included) |
| PATROL_BORO | The name of the patrol borough in which the incident occurred |
| PD_CD | Three digit internal classification code (more granular than Key Code) |
| PD_DESC | Description of internal classification corresponding with PD code (more granular than Offense Description) |
| PREM_TYP_DESC |  Specific description of premises; grocery store, residence, street, etc. |
| RPT_DT | Date event was reported to police  |
| STATION_NAME | Transit station name |
| SUSP_AGE_GROUP | Suspect’s Age Group |
| SUSP_RACE | Suspect’s Race Description |
| SUSP_SEX | Suspect’s Sex Description |
| TRANSIT_DISTRICT | Transit district in which the offense occurred. |
| VIC_AGE_GROUP | Victim’s Age Group |
| VIC_RACE | Victim’s Race Description |
| VIC_SEX | Victim’s Sex Description (D=Business/Organization, E=PSNY/People of the State of New York, F=Female, M=Male) |
| X_COORD_CD | X-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104) |
| Y_COORD_CD | Y-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104) |
| Latitude | Midblock Latitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326) |
| Longitude | Midblock Longitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326) |

##### Loading data

In [13]:
df = pd.read_csv('data\\NYPD_Complaint_Data_Historic.csv',
                 low_memory= False,
                #  nrows= 5,
                parse_dates= [1],
                # index_col= 0
                )

# check what kind of date object cmplnt_fr_dt is
# then use functions to extract each component for analysis

In [3]:
df.shape

(7825499, 35)

In [120]:
df.index

RangeIndex(start=0, stop=7825499, step=1)

In [10]:
df['date'] = pd.to_datetime(df['CMPLNT_FR_DT'])

In [11]:
df['CMPLNT_FR_DT']

CMPLNT_NUM
506547392   2018-03-29
629632833   2018-02-06
787203902   2018-11-21
280364018   2018-06-09
985800320   2018-11-10
Name: CMPLNT_FR_DT, dtype: datetime64[ns]

In [12]:
df.DATE

CMPLNT_NUM
506547392   2018-03-29
629632833   2018-02-06
787203902   2018-11-21
280364018   2018-06-09
985800320   2018-11-10
Name: DATE, dtype: datetime64[ns]

### Analysis

##### removing nulls and unnecessary columns

In [ ]:
# Check percentage of nulls
# df.isnull().sum() * 100 / len(df)

In [14]:
drop_cols = ['HADEVELOPT',  'STATION_NAME', 'HOUSING_PSA', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX',
            'TRANSIT_DISTRICT', 'STATION_NAME', 'VIC_AGE_GROUP', 'LOC_OF_OCCUR_DESC', 'JURIS_DESC']

df.drop(labels= drop_cols, axis= 1, inplace= True)

# dropping nulls, because dropna does not work in this case
df = df.loc[~df['CMPLNT_FR_DT'].isnull()]

##### Extract column data types to change them if needed

In [ ]:
for col in df.columns:
    print(f'\'{col}\':\'{df[col].dtype}\'', end = ', ')

col_types = {

}

df = df.astype(col_types, errors = 'ignore')

##### Subset data to reduce memory usage

In [15]:
A_df = df[['CMPLNT_NUM', 'OFNS_DESC', 'BORO_NM', 'CMPLNT_FR_DT']].copy(deep = True).sort_values(by= ['CMPLNT_FR_DT'])

In [16]:
A_df.head()

,CMPLNT_NUM,OFNS_DESC,BORO_NM,CMPLNT_FR_DT
6162282,688541128,OFF. AGNST PUB ORD SENSBLTY &,QUEENS,01/01/1948
806537,826810741,SEX CRIMES,STATEN ISLAND,01/01/1955
1226315,865130884,SEX CRIMES,BROOKLYN,01/01/1955
6813738,135769527,GRAND LARCENY,BROOKLYN,01/01/1964
440416,291498531,SEX CRIMES,MANHATTAN,01/01/1965


In [17]:
def extract_year(mydat):
    temp = int(str(mydat)[-4:])
    return temp if temp >= 2006 else None

def extract_month(mydat):
    temp = int(str(mydat)[:2])

    return temp

def extract_day(mydat):
    return int(str(mydat[3:5]))

# extract the year from the complaint date
# and if its before 2006, replace with None
A_df['year'] = A_df['CMPLNT_FR_DT'].apply(extract_year)
A_df['month'] = A_df['CMPLNT_FR_DT'].apply(extract_month)
A_df['day'] = A_df['CMPLNT_FR_DT'].apply(extract_day)

# drop years before 2006
A_df.dropna(inplace= True)


# A_df['year'] = pd.DatetimeIndex(A_df['CMPLNT_FR_DT']).year
# A_df['month'] = A_df['CMPLNT_FR_DT'].dt.month

In [22]:
# # Takes a lot of time
# A_df.rename(columns= {'CMPLNT_NUM': 'id', 'OFNS_DESC': 'offense', 'CMPLT_FR_DT': 'date', 'BORO_NM': 'borough'})

A_df.columns = A_df.columns.to_series().replace({'CMPLNT_NUM': 'id', 'OFNS_DESC': 'offense', 'CMPLNT_FR_DT': 'date', 'BORO_NM': 'borough'})

In [23]:
A_df.head()

,id,offense,borough,date,year,month,day
7083503,371806352,GRAND LARCENY OF MOTOR VEHICLE,BRONX,01/01/2006,2006.0,1,1
4364668,646557200,SEX CRIMES,MANHATTAN,01/01/2006,2006.0,1,1
4782863,158260094,ROBBERY,BRONX,01/01/2006,2006.0,1,1
3895184,487212855,SEX CRIMES,BROOKLYN,01/01/2006,2006.0,1,1
1336787,129185023,SEX CRIMES,QUEENS,01/01/2006,2006.0,1,1


In [25]:
# create csv for subsetted data
A_df.to_csv('data\\neighbh_crime_dist.csv', columns= ['id', 'offense', 'borough', 'date', 'year', 'month', 'day'])

#### V1: The most and least committed crimes

In [ ]:
crime_type_count = A_df.groupby(['OFNS_DESC'])['OFNS_DESC'].count().sort_values(ascending= False)

crime_type_count

##### Treemap: Crime Distribution in New York
* rebuild with law cat code

In [ ]:
# Credit to FEDI BEN MESSAOUD: https://www.kaggle.com/code/fedi1996/boston-crime-analysis-with-plotly
def treemap(categories,title,path,values):
    fig = px.treemap(categories, path=path, values=values, height=1000,
                 title=title, color_discrete_sequence = px.colors.sequential.RdBu)
    fig.data[0].textinfo = 'label+text+value'
    fig.show()

crime_count = df['OFNS_DESC'].value_counts()
vals = crime_count.values

categories = pd.DataFrame(data= crime_count.index, columns=["OFNS_DESC"])
categories['values'] = vals

treemap(categories,'Crimes in New York',['OFNS_DESC'],categories['values'])

#### V2 The most and least crime neighborhoods
- want to see how that changes every year
    - pie chart

In [27]:
# OPTION 1: use borough
q2_df = pd.read_csv('data\\neighbh_crime_dist.csv',
                 low_memory= False,
                 nrows= 500000,
                # parse_dates= [1],
                 index_col= 'id'
                ).drop('Unnamed: 0', axis= 1)


In [28]:
q2_df.head()

,offense,borough,date,year,month,day
id,,,,,,
371806352,GRAND LARCENY OF MOTOR VEHICLE,BRONX,01/01/2006,2006.0,1,1
646557200,SEX CRIMES,MANHATTAN,01/01/2006,2006.0,1,1
158260094,ROBBERY,BRONX,01/01/2006,2006.0,1,1
487212855,SEX CRIMES,BROOKLYN,01/01/2006,2006.0,1,1
129185023,SEX CRIMES,QUEENS,01/01/2006,2006.0,1,1


In [30]:
bor = ['BROOKLYN', 'MANHATTAN','BRONX', 'QUEENS', 'STATEN ISLAND']
years = [2021]


selection = q2_df[q2_df['borough'].isin(bor) | q2_df['year'].isin(years)]

vcs = selection[['borough', 'year']].value_counts().reset_index(name= 'Count').sort_values(by= ['year'])

ax = px.line(vcs, x= 'year', y= 'Count', color= 'borough', labels={'year': 'Year'}, )

# st.plotly_chart(ax)
ax.show()

In [84]:
type(pd.MultiIndex.from_arrays([['a', 'b'], ['c', 'd']]))

pandas.core.indexes.multi.MultiIndex

In [ ]:
q2_df.groupby(['BORO_NM'])['BORO_NM'].count().sort_values(ascending= False) / len(A_df) * 100

# OPTION 2: possibly use an external dataset of all precinct neighborhood locations, map to the complaint data

In [ ]:
def show_neigh_crime_pie(t_df, year):
    plt.figure()

    selection = t_df[t_df['CMPLNT_FR_YR'].isin(year)]
    ax = selection['BORO_NM'].value_counts().plot(kind= 'pie', autopct = "%1.1f%%")
    ax.set_title(f'Distribution of Crime per Borough in {year}')

    plt.show()

In [ ]:
for y in range(2006,2022, 3):
    show_neigh_crime_pie(q2_df, [y])

#### The stations with the most crimes

#### Most and Least commited crimes by season

In [ ]:
# sns.lineplot(df_, x= )